In [244]:
#%pip install ipywidgets
#%pip install voila
#%pip install voila-gridstack

In [245]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pandas.util.testing as tm
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
import ipywidgets as wdg
from IPython.display import display

In [246]:
### input data
## data filename
datafilename1 = 'sepsis_survival_primary.csv'
datafilename2 = 'sepsis_survival_study.csv'
datafilename3 = 'sepsis_survival_validation.csv'

## depended Variable column name
dependVar = 'hospital_outcome_1alive_0dead'
## exclude columns from analyze (text, urls, etc. )
excludeColumns = []
## minimum correlation coeff to assume as a key variable
minimumCorrCoef = 0.01
threshold = 0.5


In [255]:
## read CSV file, autodetect delimeters, skip spaces in names
df1 = pd.read_csv(datafilename1, sep=None, engine="python", skipinitialspace=True)
df2 = pd.read_csv(datafilename2, sep=None, engine="python", skipinitialspace=True)
df3 = pd.read_csv(datafilename3, sep=None, engine="python", skipinitialspace=True)
dfs = [df1, df2, df3]
df1.head()

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,21,1,1,1
1,20,1,1,1
2,21,1,1,1
3,77,0,1,1
4,72,0,1,1


In [286]:
for df in dfs:
    ## exclude columns
    df.drop(excludeColumns, axis ='columns', inplace = True)
    ## and drop  all NaN
    df.dropna(inplace=True)
    ## place depended var into pos 0
    poped = df.pop(dependVar)
    df.insert(0, poped.name, poped)
    #how many values are missing
    print("Total NaN: ",df.isnull().sum().sum())
          #df.isnull().sum())
        
## independed variable columns names  
independVarList = list(df.columns.values)
independVarList.remove(dependVar)
dfclean = df.copy()

Total NaN:  0
Total NaN:  0
Total NaN:  0


In [293]:
allVars=dfclean.columns
####WORK ON THIS ONE!!!!!
##PUT widgets horizontally
nxdr = wdg.Dropdown(options = allVars, value = allVars[1], 
                    description = 'X value', option = 'X var')
nBinsdr = wdg.Dropdown(options=[2,10,15,20,40], value=2, 
                    description = 'Bins number', option = 'Bin value')

def plot_histogram(x = 'age_years', nBins = 15, PlotKDE = False):    
    sns.histplot(data = dfclean[x] , bins = nBins, kde = PlotKDE)

out = wdg.interact(plot_histogram, nBins = nBinsdr,PlotKDE = False,x=nxdr)
ui = wdg.HBox([nxdr,nBinsdr])
display(ui,out)

interactive(children=(Dropdown(description='X value', index=1, options=('hospital_outcome_1alive_0dead', 'age_…

<function __main__.plot_histogram(x='age_years', nBins=15, PlotKDE=False)>

In [279]:
dftrain = dfclean.sample(frac=0.8, random_state = 200) #training data set (80%)
dftest = dfclean.drop(dftrain.index) #testing data set (20%)

## Regular regressional analise through all possible variables

In [232]:
def HeatmapCorr(df): 
    sns.set(rc = {'figure.figsize':(15,8)})
    sns.set_theme(style="white")
    corr = df.corr()
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))
    # Custom colormap
    cmap = sns.diverging_palette(275, 150, s=90, l=50, n=9, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap,  linewidths=0.3, cbar_kws={"shrink":0.5})

In [233]:
#HeatmapCorr(df1)

In [234]:
def GetKeyVariables(df):
    parCorr = pd.DataFrame(df.corr() )
    n = len(parCorr.columns)
    keyVars = pd.DataFrame(columns=['key', 'val'])
    ## depended variable moved to index 0
    i = 0
    for j in range( n):
        if j >= i:
            keyVars = keyVars.append({'key':parCorr.columns[j],'val':parCorr.iloc[i, j]}, ignore_index=True)
    ## sort key vars by value
    keyVars.pop(df.columns[0])
    keyVars.sort_values(by='val', key=abs, ascending=False, inplace=True)
    return keyVars

#display(GetKeyVariables(dftrain[[dependVar, *set1]]))
#display(GetKeyVariables(dftrain[[dependVar, *set2]]))
#display(GetKeyVariables(dftrain[[dependVar, *set3]]))
#display(GetKeyVariables(dftrain[[dependVar, *set4]]))
#display(GetKeyVariables(dftrain[[dependVar, *set5]]))
#display(GetKeyVariables(dftrain))


In [235]:
#for1 = dftrain.columns[4]+'~'+dftrain.columns[4]

In [236]:
#mod = smf.ols(formula=for1, data = dftrain)
#res = mod.fit()
#res.summary()

In [237]:
def LogRegModel(independVarList):
    indepVars = ''
    for i in range(0,len(independVarList)-1):
        indepVars+=independVarList[i]
        indepVars+=" + "
    indepVars+=independVarList[-1]
    depVar = dependVar
    model = smf.logit(formula = depVar+' ~ '+ indepVars,data = dftrain)
    res = model.fit()
    print(depVar,'~',indepVars,':',
              round(res.prsquared,5))#McFadden's pseudo-R-squared.
    """
    inSample = pd.DataFrame({'prob':res.predict(dftest[independVarList])}) #probabilities 
    inSample['predLabel'] = (inSample['prob'] > threshold).astype(int) #labels based on probabilities
    inSample.head()
    confMatrix = pd.crosstab(inSample['predLabel'],dftest[depVar])
    #sns.heatmap(confMatrix, annot = True)                  
    #plt.show()
    #calculation of BA 
    BA=0
    TN = confMatrix.loc[0,0]
    FN = confMatrix.loc[0,1]
    FP = confMatrix.loc[1,0]
    TP = confMatrix.loc[1,1]
    TPR = TP/(TP + FN)
    TNR = TN/(TN + FP)
    BA = (TPR + TNR)/2
    print(f'BA = {BA:0.4f}')
    """
def LogRegModelSqrt(independVarList):
    indepVars = ''
    for i in range(0,len(independVarList)-1):
        indepVars+='np.power('+independVarList[i]+',2)'
        indepVars+=" + "
    indepVars+='np.power('+independVarList[-1]+',2)'
    depVar = dependVar
    model = smf.logit(formula = depVar+' ~ '+ indepVars,data = dftrain)
    res = model.fit()
    print(depVar,'~',indepVars,':',
              round(res.prsquared,5))#McFadden's pseudo-R-squared.

In [238]:
LogRegModel(independVarList)
LogRegModel(independVarList[0:1])
LogRegModel(independVarList[1:2])
LogRegModel(independVarList[2:3])

Optimization terminated successfully.
         Current function value: 0.403407
         Iterations 6
hospital_outcome_1alive_0dead ~ age_years + sex_0male_1female + episode_number : 0.12338
Optimization terminated successfully.
         Current function value: 0.459467
         Iterations 6
hospital_outcome_1alive_0dead ~ age_years : 0.00156
Optimization terminated successfully.
         Current function value: 0.458175
         Iterations 6
hospital_outcome_1alive_0dead ~ sex_0male_1female : 0.00437
Optimization terminated successfully.
         Current function value: 0.405959
         Iterations 6
hospital_outcome_1alive_0dead ~ episode_number : 0.11783


In [239]:
LogRegModelSqrt(independVarList)
LogRegModelSqrt(independVarList[0:1])
LogRegModelSqrt(independVarList[1:2])
LogRegModelSqrt(independVarList[2:3])

Optimization terminated successfully.
         Current function value: 0.403406
         Iterations 6
hospital_outcome_1alive_0dead ~ np.power(age_years,2) + np.power(sex_0male_1female,2) + np.power(episode_number,2) : 0.12338
Optimization terminated successfully.
         Current function value: 0.459445
         Iterations 6
hospital_outcome_1alive_0dead ~ np.power(age_years,2) : 0.00161
Optimization terminated successfully.
         Current function value: 0.458175
         Iterations 6
hospital_outcome_1alive_0dead ~ np.power(sex_0male_1female,2) : 0.00437
Optimization terminated successfully.
         Current function value: 0.405959
         Iterations 6
hospital_outcome_1alive_0dead ~ np.power(episode_number,2) : 0.11783
